In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel


class SIMCSEmodel(nn.Module):
    def __init__(self):
        super(SIMCSEmodel, self).__init__()

        self.model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")

        def forward(self, x):
            embeddings = self.model(**x, output_hidden_states=True, return_dict=True).pooler_output
            return embeddings

class DeepSC_SIMCSE2FC(nn.Module):
    def __init__(self, vocab_size):
        super(DeepSC_SIMCSE2FC, self).__init__()
        
        self.encoder = SIMCSEmodel()
        self.dense = nn.Linear(768, vocab_size)


In [7]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

# Import our models. The package will take care of downloading the models automatically
tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")
model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")

# Tokenize input texts
texts = [
    "There's a kid on a skateboard.",
    # "A kid is skateboarding.",
    # "A kid is inside the house."
]
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
print(inputs)

# Get the embeddings
with torch.no_grad():
    embeddings = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output
    print(embeddings.shape)

# Calculate cosine similarities
# Cosine similarities are in [-1, 1]. Higher means more similar
# cosine_sim_0_1 = 1 - cosine(embeddings[0], embeddings[1])
# cosine_sim_0_2 = 1 - cosine(embeddings[0], embeddings[2])

# print("Cosine similarity between \"%s\" and \"%s\" is: %.3f" % (texts[0], texts[1], cosine_sim_0_1))
# print("Cosine similarity between \"%s\" and \"%s\" is: %.3f" % (texts[0], texts[2], cosine_sim_0_2))

{'input_ids': tensor([[  101,  2045,  1005,  1055,  1037,  4845,  2006,  1037, 17260,  6277,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
torch.Size([1, 768])
